# Word Embeddings : le modèle Word2Vec

## Imports

In [21]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [22]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [23]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [24]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [25]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [26]:
len(bigram_phrases.vocab.keys())

1877910

Prenons une clé au hasard :

In [27]:
key_= list(bigram_phrases.vocab.keys())[200]
print(key_)

de_nos


Le dictionnaire indique le score de cette coocurrence :

In [28]:
bigram_phrases.vocab[key_]

1735

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [29]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [30]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [31]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [32]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [33]:
print(corpus[:100])

[['samedi_22', 'mars_1930', '.'], ['<<', 'le', 'drapeau_rouge', '>>', 'u', 'fleches_rouges', 'u', 'les', 'faits', 'internationaux', 'de', 'la', 'semaine', 'les', 'lauriers', 'de', 'm', '.', 'sinzot', 'empechent', 'm', '.', 'branquart', 'de', 'dormir', '.'], ['m', '.', 'sinzot', 'est', ',', 'evidemment', ',', 'un_homme', '<<', 'de', 'stoc', '>>.'], ['//', 'sait', 'rugir', ',', 'mugir', 'a', 'lui_seul', ',', 'mieux', 'que', 'toute', 'une', 'menagerie', '.'], ['fl', 'a', 'employe', 'ses_talents', ',', 'a', 'la', 'chambre', ',', 'pour', ',', 'denoncer', 'les', 'communistes', 'en', 'general', 'ct', 'notre_camarade', ',', '[', 'acquemotte', ',', 'en_particulier', ',', 'comme', 'des', '<<', 'pilleurs', 'dc', 'cadavres', '>>.'], ['tout_cela', 'a', 'propos', 'de', 'la', 'scandaleuse', 'parade', 'du', 'consul', 'fasciste', 'polonais', ',', 'des', 'administrateurs', 'du', 'charbonnage', 'du', 'cazier', 'et', 'du_college', '<<', 'socialiste', '>>', 'de', 'marcinelle', 'sur', 'les', 'tombes', 'de',

## Entrainement d'un modèle Word2Vec sur ce corpus

In [70]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=32, # Le nombre de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=3, # La taille du "contexte", ici 3 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: total: 5.84 s
Wall time: 12.6 s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [71]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [72]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [77]:
model.wv["crise"]

array([ 0.76995975, -2.18457   ,  2.8102584 , -3.0724473 ,  0.261696  ,
        1.3371208 , -1.7407337 ,  1.0727575 ,  2.8017182 ,  3.05948   ,
        1.3935755 ,  0.7363242 , -1.4966685 , -0.06150797,  3.9841597 ,
        1.7362409 ,  0.81300026,  1.8122014 , -0.13274147,  2.183218  ,
        0.3567307 , -4.192907  ,  3.5707695 ,  0.4806828 ,  0.7502151 ,
        2.1468632 ,  0.0659516 , -0.1873255 ,  2.77066   ,  0.05602318,
        2.2449722 , -1.0648581 ], dtype=float32)

### Calculer la similarité entre deux termes

In [78]:
model.wv.similarity("crise", "depression")

0.82916397

### Chercher les mots les plus proches d'un terme donné

In [79]:
model.wv.most_similar("crise", topn=10)

[('crise_economique', 0.9322041273117065),
 ('prosperite', 0.8320592045783997),
 ('crise_mondiale', 0.8301627039909363),
 ('depression', 0.8291639685630798),
 ('surproduction', 0.8280938267707825),
 ('devaluation', 0.8256010413169861),
 ('chomage', 0.8087950348854065),
 ('misere', 0.8083654642105103),
 ('situation', 0.797191858291626),
 ('crise_actuelle', 0.792048990726471)]

### Faire des recherches complexes à travers l'espace vectoriel

In [80]:
print(model.wv.most_similar(positive=['crise', 'depression'], negative=['politique']))

[('hausse', 0.8495281338691711), ('baisse', 0.8397588729858398), ('regression', 0.801382839679718), ('proportion', 0.7890205383300781), ('semaine', 0.7838152647018433), ('devaluation', 0.7817928194999695), ('recolte', 0.7789984941482544), ('diminution', 0.7749210000038147), ('surproduction', 0.7723634243011475), ('duree', 0.7693139910697937)]
